## Install obspy

Remember to click "Runtime" then "Restart runtime", otherwise obspy may report some error.

See details in this [issue](https://github.com/obspy/obspy/issues/2547)

Note: Please only use the QuakeFlow API for debugging and testing on small datasets. Do not run large jobs using the QuakeFlow API. The computational cost can be high for us.

In [5]:
try:
    import obspy
except:
    !pip install obspy

In [6]:
from collections import defaultdict
import numpy as np
import pandas as pd
import time
import requests
import json
import obspy
from obspy.clients.fdsn import Client

# from google.colab import data_table
# data_table.enable_dataframe_formatter()

## 1. Configuration

In [7]:
region_name = "Indonesia_demo"
center = (107.67, -7.19)  # Koordinat pusat Indonesia
horizontal_degree = 50.0  # Jangkauan horizontal dalam derajat
vertical_degree = 50.0  # Jangkauan vertikal dalam derajat
starttime = obspy.UTCDateTime("2024-09-18T02:00:00")
endtime = obspy.UTCDateTime("2024-09-18T03:00:00")
client = "http://geofon.gfz-potsdam.de"  # Client untuk data seismik
network_list = ["GE"]  # Contoh jaringan internasional di Indonesia
channel_list = "BHZ, BHN, BHE"  # Daftar kanal umum untuk stasiun geofon

config = {}
config["region"] = region_name
config["center"] = center
config["xlim_degree"] = [center[0] - horizontal_degree / 2, center[0] + horizontal_degree / 2]
config["ylim_degree"] = [center[1] - vertical_degree / 2, center[1] + vertical_degree / 2]
config["starttime"] = starttime.datetime.isoformat()
config["endtime"] = endtime.datetime.isoformat()
config["networks"] = network_list
config["channels"] = channel_list
config["client"] = client


## 2. Download event information

In [8]:
events = Client("IRIS").get_events(
    starttime=config["starttime"],
    endtime=config["endtime"],
    minlongitude=config["xlim_degree"][0],
    maxlongitude=config["xlim_degree"][1],
    minlatitude=config["ylim_degree"][0],
    maxlatitude=config["ylim_degree"][1],
)

In [9]:
events[0]["origins"][0].time

2024-09-18T02:41:06.667000Z

## 3. Download station information

In [10]:
stations = Client(config["client"]).get_stations(
    network=",".join(config["networks"]),
    station="*",
    starttime=config["starttime"],
    endtime=config["endtime"],
    minlongitude=config["xlim_degree"][0],
    maxlongitude=config["xlim_degree"][1],
    minlatitude=config["ylim_degree"][0],
    maxlatitude=config["ylim_degree"][1],
    channel=config["channels"],
    level="response",
)

In [11]:
stations

Inventory created at 2024-12-16T15:26:41.177635Z
	Sending institution: SeisComP (GFZ)
	Contains:
		Networks (1):
			GE
		Stations (20):
			GE.BBJI (GEOFON Station GEOFON Station Bungbulang, Garut, Java)
			GE.BKB (GEOFON Station GEOFON Station Balikpapan, Kalimantan)
			GE.BKNI (GEOFON Station GEOFON Station Bangkinang, Sumatra)
			GE.BNDI (GEOFON Station Bandaneira, Indonesia)
			GE.FAKI (GEOFON Station GEOFON Station Fak Fak, Irian Jaya)
			GE.GSI (GEOFON Station Gunungsitoli, Nias)
			GE.JAGI (GEOFON Station GEOFON Station Jajag, Java)
			GE.LHMI (GEOFON Station GEOFON Station Lhokseumave, Sumatra)
			GE.LUWI (GEOFON Station GEOFON Station Lhokseumave, Sulawesi)
			GE.MMRI (GEOFON Station GEOFON Station Maumere, Flores)
			GE.MNAI (GEOFON Station GEOFON Station Manna, Sumatra)
			GE.PLAI (GEOFON Station Plampang, Sumbawa, Indonesia)
			GE.PMBI (GEOFON Station Palembang, Sumatra)
			GE.SANI (GEOFON Station Sanana, Moluccas, Indonesia)
			GE.SAUI (GEOFON Station Saumlaki, Tanimbar, In

## 3.1 Convert station information into csv

In [12]:
station_locs = defaultdict(dict)
for network in stations:
    for station in network:
        for chn in station:
            sid = f"{network.code}.{station.code}.{chn.location_code}.{chn.code[:-1]}"
            if sid in station_locs:
                station_locs[sid]["component"] += f",{chn.code[-1]}"
                station_locs[sid]["response"] += f",{chn.response.instrument_sensitivity.value:.2f}"
            else:
                component = f"{chn.code[-1]}"
                response = f"{chn.response.instrument_sensitivity.value:.2f}"
                dtype = chn.response.instrument_sensitivity.input_units.lower()
                tmp_dict = {}
                tmp_dict["longitude"], tmp_dict["latitude"], tmp_dict["elevation_m"] = (
                    chn.longitude,
                    chn.latitude,
                    chn.elevation,
                )
                tmp_dict["component"], tmp_dict["response"], tmp_dict["unit"] = component, response, dtype
                station_locs[sid] = tmp_dict

station_locs = pd.DataFrame.from_dict(station_locs, orient='index')
station_locs["station_id"] = station_locs.index

## 4. Download waveform

In [13]:
client = Client(config["client"])
interval = 3600 #s
# interval = 3600 #s
interval_start = 60

# for event in events:
def downlad(event, stations):
    starttime = event["origins"][0].time
    endtime = starttime + interval

    max_retry = 20
    stream = obspy.Stream()
    num_sta = 0
    for network in stations:
        for station in network:
            print(f"********{network.code}.{station.code}********")
            retry = 0
            while retry < max_retry:
                try:
                    tmp = client.get_waveforms(
                        network.code, station.code, "*", config["channels"], starttime, endtime
                    )
                    for trace in tmp:
                        if trace.stats.sampling_rate != 100:
                            # print(trace)
                            trace = trace.interpolate(100, method="linear")
                    #      trace = trace.detrend("spline", order=2, dspline=5*trace.stats.sampling_rate)
                    #      stream.append(trace)
                    stream += tmp
                    num_sta += len(tmp)
                    break
                except Exception as err:
                    print("Error {}.{}: {}".format(network.code, station.code, err))
                    message = "No data available for request."
                    if str(err)[: len(message)] == message:
                        break
                    retry += 1
                    time.sleep(5)
                    continue
            if retry == max_retry:
                print(f"{station.code}: MAX {max_retry} retries reached : {network.code}.{station.code}")

    # stream.attach_response(stations)
    # stream = stream.remove_sensitivity()
    return stream

In [14]:
mseed = downlad(events[0], stations)

********GE.BBJI********
********GE.BKB********
********GE.BKNI********
Error GE.BKNI: No data available for request.
HTTP Status code: 204
Detailed response of server:


********GE.BNDI********
********GE.FAKI********
********GE.GSI********
********GE.JAGI********
********GE.LHMI********
********GE.LUWI********
********GE.MMRI********
********GE.MNAI********
********GE.PLAI********
********GE.PMBI********
********GE.SANI********
********GE.SAUI********
Error GE.SAUI: No data available for request.
HTTP Status code: 204
Detailed response of server:


********GE.SMRI********
********GE.SOEI********
********GE.TNTI********
********GE.TOLI2********
********GE.UGM********
Error GE.UGM: No data available for request.
HTTP Status code: 204
Detailed response of server:




## 5. Convert waveform to numpy

In [15]:
sampling_rate = 100
n_channel = 3
dtype = "float32"
amplitude = True
remove_resp = True

def convert_mseed(mseed, station_locs):
    try:
        mseed = mseed.detrend("spline", order=2, dspline=5 * mseed[0].stats.sampling_rate)
    except:
        logging.error(f"Error: spline detrend failed at file {fname}")
        mseed = mseed.detrend("demean")
    mseed = mseed.merge(fill_value=0)
    starttime = min([st.stats.starttime for st in mseed])
    endtime = max([st.stats.endtime for st in mseed])
    mseed = mseed.trim(starttime, endtime, pad=True, fill_value=0)

    for i in range(len(mseed)):
        if mseed[i].stats.sampling_rate != sampling_rate:
            logging.warning(
                f"Resampling {mseed[i].id} from {mseed[i].stats.sampling_rate} to {sampling_rate} Hz"
            )
            mseed[i] = mseed[i].interpolate(sampling_rate, method="linear")

    order = ['3', '2', '1', 'E', 'N', 'Z']
    order = {key: i for i, key in enumerate(order)}
    comp2idx = {"3": 0, "2": 1, "1": 2, "E": 0, "N": 1, "Z": 2}

    nsta = len(station_locs)
    nt = max(len(mseed[i].data) for i in range(len(mseed)))
    data = []
    station_id = []
    t0 = []
    for i in range(nsta):
        trace_data = np.zeros([nt, n_channel], dtype=dtype)
        empty_station = True
        # sta = station_locs.iloc[i]["station"]
        sta = station_locs.index[i]
        comp = station_locs.iloc[i]["component"].split(",")
        if remove_resp:
            resp = station_locs.iloc[i]["response"].split(",")
            # resp = station_locs.iloc[i]["response"]

        for j, c in enumerate(sorted(comp, key=lambda x: order[x[-1]])):

            resp_j = float(resp[j])
            if len(comp) != 3:  ## less than 3 component
                j = comp2idx[c]

            if len(mseed.select(id=sta + c)) == 0:
                print(f"Empty trace: {sta+c} {starttime}")
                continue
            else:
                empty_station = False

            tmp = mseed.select(id=sta + c)[0].data.astype(dtype)
            trace_data[: len(tmp), j] = tmp[:nt]

            if station_locs.iloc[i]["unit"] == "m/s**2":
                tmp = mseed.select(id=sta + c)[0]
                tmp = tmp.integrate()
                tmp = tmp.filter("highpass", freq=1.0)
                tmp = tmp.data.astype(dtype)
                trace_data[: len(tmp), j] = tmp[:nt]
            elif station_locs.iloc[i]["unit"] == "m/s":
                tmp = mseed.select(id=sta + c)[0].data.astype(dtype)
                trace_data[: len(tmp), j] = tmp[:nt]
            else:
                print(
                    f"Error in {station_locs.iloc[i]['station']}\n{station_locs.iloc[i]['unit']} should be m/s**2 or m/s!"
                )

            if remove_resp:
                trace_data[:, j] /= resp_j

        if not empty_station:
            data.append(trace_data)
            station_id.append(sta)
            t0.append(starttime.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3])

    data = np.stack(data)

    meta = {"data": data, "t0": t0, "station_id": station_id, "fname": station_id}


    return meta

In [16]:
meta = convert_mseed(mseed, station_locs)

Empty trace: GE.BKNI..BHE 2024-09-18T02:41:06.669536Z
Empty trace: GE.BKNI..BHN 2024-09-18T02:41:06.669536Z
Empty trace: GE.BKNI..BHZ 2024-09-18T02:41:06.669536Z
Empty trace: GE.SAUI..BHE 2024-09-18T02:41:06.669536Z
Empty trace: GE.SAUI..BHN 2024-09-18T02:41:06.669536Z
Empty trace: GE.SAUI..BHZ 2024-09-18T02:41:06.669536Z
Empty trace: GE.UGM..BHE 2024-09-18T02:41:06.669536Z
Empty trace: GE.UGM..BHN 2024-09-18T02:41:06.669536Z
Empty trace: GE.UGM..BHZ 2024-09-18T02:41:06.669536Z


## 6. Pick P/S picks using PhaseNet

In [17]:
PHASENET_API_URL = "http://34.30.199.8:8000"

batch = 4
phasenet_picks = []
for j in range(0, len(meta["station_id"]), batch):
    req = {"id": meta['station_id'][j:j+batch],
        "timestamp": meta["t0"][j:j+batch],
        "vec": meta["data"][j:j+batch].tolist()}

# print(meta["data"][0])

    resp = requests.post(f'{PHASENET_API_URL}/predict', json=req)
    print(resp.status_code)
    print(resp.text)

    phasenet_picks.extend(resp.json())

print('PhaseNet picks', pd.DataFrame(phasenet_picks))


200
[{"station_id":"GE.BBJI..BH","phase_time":"2024-09-18T02:41:12.889","phase_score":0.842,"phase_type":"P","dt":0.01,"phase_amplitude":0.005412620026618242},{"station_id":"GE.BBJI..BH","phase_time":"2024-09-18T02:43:52.929","phase_score":0.433,"phase_type":"P","dt":0.01,"phase_amplitude":6.37675475445576e-05},{"station_id":"GE.BBJI..BH","phase_time":"2024-09-18T02:45:27.619","phase_score":0.899,"phase_type":"P","dt":0.01,"phase_amplitude":1.3554576071328484e-05},{"station_id":"GE.BBJI..BH","phase_time":"2024-09-18T02:46:39.539","phase_score":0.836,"phase_type":"P","dt":0.01,"phase_amplitude":1.0399807251815218e-05},{"station_id":"GE.BBJI..BH","phase_time":"2024-09-18T02:47:40.049","phase_score":0.728,"phase_type":"P","dt":0.01,"phase_amplitude":3.3307564990536775e-06},{"station_id":"GE.BBJI..BH","phase_time":"2024-09-18T02:48:26.449","phase_score":0.901,"phase_type":"P","dt":0.01,"phase_amplitude":9.718434739625081e-05},{"station_id":"GE.BBJI..BH","phase_time":"2024-09-18T02:49:41.21

In [31]:
pd.DataFrame(phasenet_picks)

,station_id,phase_time,phase_score,phase_type,dt,phase_amplitude
0,GE.BBJI..BH,2024-09-18T02:41:12.889,0.842,P,0.01,5.412620e-03
1,GE.BBJI..BH,2024-09-18T02:43:52.929,0.433,P,0.01,6.376755e-05
2,GE.BBJI..BH,2024-09-18T02:45:27.619,0.899,P,0.01,1.355458e-05
3,GE.BBJI..BH,2024-09-18T02:46:39.539,0.836,P,0.01,1.039981e-05
4,GE.BBJI..BH,2024-09-18T02:47:40.049,0.728,P,0.01,3.330756e-06
...,...,...,...,...,...,...
237,GE.TOLI2..BH,2024-09-18T03:21:56.669,0.505,S,0.01,2.172834e-07
238,GE.TOLI2..BH,2024-09-18T03:22:48.589,0.547,S,0.01,3.097030e-07
239,GE.TOLI2..BH,2024-09-18T03:23:40.269,0.485,S,0.01,2.467740e-07
240,GE.TOLI2..BH,2024-09-18T03:23:48.169,0.335,S,0.01,1.933812e-07


In [32]:
# prompt: export phasenet_picks to excel

import pandas as pd

# Assuming 'phasenet_picks' is already defined from the previous code

# Create a Pandas DataFrame from the phasenet_picks list
phasenet_df = pd.DataFrame(phasenet_picks)

# Export the DataFrame to an Excel file
phasenet_df.to_excel("phasenet_picks.xlsx", index=False)

In [18]:
type([meta['station_id'][j:j+batch]])

list

In [19]:
meta["t0"][j:j+batch]

['2024-09-18T02:41:06.669']

In [20]:
meta["data"][j:j+batch].shape

(1, 360001, 3)

In [21]:
stations_json = station_locs.to_dict(orient="records")


In [22]:
stations_json

[{'longitude': 107.65,
  'latitude': -7.462603,
  'elevation_m': 615.0,
  'component': 'N,E,Z',
  'response': '629145000.00,629145000.00,629145000.00',
  'unit': 'm/s',
  'station_id': 'GE.BBJI..BH'},
 {'longitude': 116.9048,
  'latitude': -1.1073,
  'elevation_m': 110.0,
  'component': 'N,E,Z',
  'response': '629145000.00,629145000.00,629145000.00',
  'unit': 'm/s',
  'station_id': 'GE.BKB..BH'},
 {'longitude': 101.0396,
  'latitude': 0.3262,
  'elevation_m': 65.0,
  'component': 'N,E,Z',
  'response': '629145000.00,629145000.00,629145000.00',
  'unit': 'm/s',
  'station_id': 'GE.BKNI..BH'},
 {'longitude': 129.9045,
  'latitude': -4.5224,
  'elevation_m': 16.0,
  'component': 'N,E,Z',
  'response': '629145000.00,629145000.00,629145000.00',
  'unit': 'm/s',
  'station_id': 'GE.BNDI..BH'},
 {'longitude': 132.24889,
  'latitude': -2.91925,
  'elevation_m': 136.0,
  'component': 'N,E,Z',
  'response': '629145000.00,629145000.00,629145000.00',
  'unit': 'm/s',
  'station_id': 'GE.FAKI..BH'

In [23]:
phasenet_picks

[{'station_id': 'GE.BBJI..BH',
  'phase_time': '2024-09-18T02:41:12.889',
  'phase_score': 0.842,
  'phase_type': 'P',
  'dt': 0.01,
  'phase_amplitude': 0.005412620026618242},
 {'station_id': 'GE.BBJI..BH',
  'phase_time': '2024-09-18T02:43:52.929',
  'phase_score': 0.433,
  'phase_type': 'P',
  'dt': 0.01,
  'phase_amplitude': 6.37675475445576e-05},
 {'station_id': 'GE.BBJI..BH',
  'phase_time': '2024-09-18T02:45:27.619',
  'phase_score': 0.899,
  'phase_type': 'P',
  'dt': 0.01,
  'phase_amplitude': 1.3554576071328484e-05},
 {'station_id': 'GE.BBJI..BH',
  'phase_time': '2024-09-18T02:46:39.539',
  'phase_score': 0.836,
  'phase_type': 'P',
  'dt': 0.01,
  'phase_amplitude': 1.0399807251815218e-05},
 {'station_id': 'GE.BBJI..BH',
  'phase_time': '2024-09-18T02:47:40.049',
  'phase_score': 0.728,
  'phase_type': 'P',
  'dt': 0.01,
  'phase_amplitude': 3.3307564990536775e-06},
 {'station_id': 'GE.BBJI..BH',
  'phase_time': '2024-09-18T02:48:26.449',
  'phase_score': 0.901,
  'phase_ty

In [24]:
picks = pd.DataFrame(phasenet_picks)
picks

,station_id,phase_time,phase_score,phase_type,dt,phase_amplitude
0,GE.BBJI..BH,2024-09-18T02:41:12.889,0.842,P,0.01,5.412620e-03
1,GE.BBJI..BH,2024-09-18T02:43:52.929,0.433,P,0.01,6.376755e-05
2,GE.BBJI..BH,2024-09-18T02:45:27.619,0.899,P,0.01,1.355458e-05
3,GE.BBJI..BH,2024-09-18T02:46:39.539,0.836,P,0.01,1.039981e-05
4,GE.BBJI..BH,2024-09-18T02:47:40.049,0.728,P,0.01,3.330756e-06
...,...,...,...,...,...,...
237,GE.TOLI2..BH,2024-09-18T03:21:56.669,0.505,S,0.01,2.172834e-07
238,GE.TOLI2..BH,2024-09-18T03:22:48.589,0.547,S,0.01,3.097030e-07
239,GE.TOLI2..BH,2024-09-18T03:23:40.269,0.485,S,0.01,2.467740e-07
240,GE.TOLI2..BH,2024-09-18T03:23:48.169,0.335,S,0.01,1.933812e-07


In [25]:
# Assuming 'picks' is your DataFrame
# Step 1: Filter rows where 'phase_amplitude' is NaN
nan_amplitude_rows = picks[picks['phase_amplitude'].isna()]

# Step 2: Check if 'phase_type' is 'P' for all those rows
all_nan_amplitude_are_P = (nan_amplitude_rows['phase_type'] == 'P').all()

if all_nan_amplitude_are_P:
    print("All NaN values in 'phase_amplitude' occur when 'phase_type' is 'P'")
else:
    print("There are NaN values in 'phase_amplitude' that do not occur when 'phase_type' is 'P'")

All NaN values in 'phase_amplitude' occur when 'phase_type' is 'P'


## 7. Associate picks using GaMMA

In [26]:
# GAMMA_API_URL = "http://gamma.quakeflow.com"
GAMMA_API_URL = "http://34.123.47.103:8000"

# stations_json = json.loads(station_locs.to_json(orient="records"))
# stations_json = station_locs.to_dict(orient="records")
# config_gamma = {'xlim_degree': config["xlim_degree"],
#                 'ylim_degree': config["ylim_degree"],
#                 'z(km)': [0, 41],
#                 'use_amplitude': False}

# phasenet_picks = pd.DataFrame(phasenet_picks)
# phasenet_picks["id"] = phasenet_picks["station_id"]
# phasenet_picks["timestamp"] = phasenet_picks["phase_time"]
# phasenet_picks["type"] = phasenet_picks["phase_type"]
# phasenet_picks["prob"] = phasenet_picks["phase_score"]
# phasenet_picks = json.loads(phasenet_picks.to_json(orient="records"))
result = requests.post(f'{GAMMA_API_URL}/predict', json= {"picks": phasenet_picks,
                                                          "stations": stations_json,
                                                          "config": {"use_amplitude": False,
                                                                     "min_picks_per_eq": 5,
                                                                      "min_p_picks_per_eq": 0,
                                                                      "min_s_picks_per_eq": 0,
                                                                      "max_sigma11": 10.0,
                                                                      "max_sigma22": 1.0,
                                                                      "max_sigma12": 1.0,}})
result = result.json()
print(result)


# events_gamma = result["events"]
# picks_gamma = result["picks"]
# print("GaMMA catalog:")
# display(pd.DataFrame(events_gamma))
# print("GaMMA association:")
# display(pd.DataFrame(picks_gamma))

{'events': [{'time': '2024-09-18T03:08:31.640', 'magnitude': 999, 'sigma_time': 4.792024295546496, 'sigma_amp': 0, 'cov_time_amp': 0, 'gamma_score': 10.762634474783443, 'num_picks': 5, 'num_p_picks': 3, 'num_s_picks': 2, 'event_index': 1, 'longitude': 108.10873817472368, 'latitude': -7.457718149352783, 'depth_km': 0.0}], 'picks': [{'station_id': 'GE.BBJI..BH', 'phase_time': '2024-09-18T02:41:12.889000', 'phase_score': 0.842, 'phase_type': 'P', 'dt': 0.01, 'phase_amplitude': 0.005412620026618242, 'event_index': -1, 'gamma_score': -1.0}, {'station_id': 'GE.BBJI..BH', 'phase_time': '2024-09-18T02:43:52.929000', 'phase_score': 0.433, 'phase_type': 'P', 'dt': 0.01, 'phase_amplitude': 6.37675475445576e-05, 'event_index': -1, 'gamma_score': -1.0}, {'station_id': 'GE.BBJI..BH', 'phase_time': '2024-09-18T02:45:27.619000', 'phase_score': 0.899, 'phase_type': 'P', 'dt': 0.01, 'phase_amplitude': 1.3554576071328484e-05, 'event_index': -1, 'gamma_score': -1.0}, {'station_id': 'GE.BBJI..BH', 'phase_t

In [27]:
events_gamma = result["events"]
picks_gamma = result["picks"]
print("GaMMA catalog:")
display(pd.DataFrame(events_gamma))
print("GaMMA association:")
display(pd.DataFrame(picks_gamma))

GaMMA catalog:


,time,magnitude,sigma_time,sigma_amp,cov_time_amp,gamma_score,num_picks,num_p_picks,num_s_picks,event_index,longitude,latitude,depth_km
0,2024-09-18T03:08:31.640,999,4.792024,0,0,10.762634,5,3,2,1,108.108738,-7.457718,0.0


GaMMA association:


,station_id,phase_time,phase_score,phase_type,dt,phase_amplitude,event_index,gamma_score
0,GE.BBJI..BH,2024-09-18T02:41:12.889000,0.842,P,0.01,5.412620e-03,-1,-1.0
1,GE.BBJI..BH,2024-09-18T02:43:52.929000,0.433,P,0.01,6.376755e-05,-1,-1.0
2,GE.BBJI..BH,2024-09-18T02:45:27.619000,0.899,P,0.01,1.355458e-05,-1,-1.0
3,GE.BBJI..BH,2024-09-18T02:46:39.539000,0.836,P,0.01,1.039981e-05,-1,-1.0
4,GE.BBJI..BH,2024-09-18T02:47:40.049000,0.728,P,0.01,3.330756e-06,-1,-1.0
...,...,...,...,...,...,...,...,...
237,GE.TOLI2..BH,2024-09-18T03:21:56.669000,0.505,S,0.01,2.172834e-07,-1,-1.0
238,GE.TOLI2..BH,2024-09-18T03:22:48.589000,0.547,S,0.01,3.097030e-07,-1,-1.0
239,GE.TOLI2..BH,2024-09-18T03:23:40.269000,0.485,S,0.01,2.467740e-07,-1,-1.0
240,GE.TOLI2..BH,2024-09-18T03:23:48.169000,0.335,S,0.01,1.933812e-07,-1,-1.0


In [28]:
phasenet_picks

[{'station_id': 'GE.BBJI..BH',
  'phase_time': '2024-09-18T02:41:12.889',
  'phase_score': 0.842,
  'phase_type': 'P',
  'dt': 0.01,
  'phase_amplitude': 0.005412620026618242},
 {'station_id': 'GE.BBJI..BH',
  'phase_time': '2024-09-18T02:43:52.929',
  'phase_score': 0.433,
  'phase_type': 'P',
  'dt': 0.01,
  'phase_amplitude': 6.37675475445576e-05},
 {'station_id': 'GE.BBJI..BH',
  'phase_time': '2024-09-18T02:45:27.619',
  'phase_score': 0.899,
  'phase_type': 'P',
  'dt': 0.01,
  'phase_amplitude': 1.3554576071328484e-05},
 {'station_id': 'GE.BBJI..BH',
  'phase_time': '2024-09-18T02:46:39.539',
  'phase_score': 0.836,
  'phase_type': 'P',
  'dt': 0.01,
  'phase_amplitude': 1.0399807251815218e-05},
 {'station_id': 'GE.BBJI..BH',
  'phase_time': '2024-09-18T02:47:40.049',
  'phase_score': 0.728,
  'phase_type': 'P',
  'dt': 0.01,
  'phase_amplitude': 3.3307564990536775e-06},
 {'station_id': 'GE.BBJI..BH',
  'phase_time': '2024-09-18T02:48:26.449',
  'phase_score': 0.901,
  'phase_ty

## Optional: Run PhaseNet and GaMMA together

In [29]:
# PHASENET_API_URL = "http://test.quakeflow.com:8001"

# req = {"id": meta["station_id"],
#        "timestamp": meta["t0"],
#        "vec": meta["data"].squeeze().tolist(),
#        "stations": stations_json,
#        "config": config_gamma}

# resp = requests.post(f'{PHASENET_API_URL}/predict_phasenet2gamma2ui', json=req)
# print(resp.json())
# result = resp.json()
# catalog_gamma = result["catalog"]
# picks_gamma = result["picks"]
# print("Catalog:")
# display(pd.DataFrame(catalog_gamma))
# print("Association:")
# display(pd.DataFrame(picks_gamma))

## Compare with official catalog

In [30]:
event = events[0]
print(event.origins[0])
print(event.magnitudes[0])

Origin
	   resource_id: ResourceIdentifier(id="smi:service.iris.edu/fdsnws/event/1/query?originid=50748472")
	          time: UTCDateTime(2024, 9, 18, 2, 41, 6, 667000)
	     longitude: 107.5693
	      latitude: -7.2397
	         depth: 10000.0
	 creation_info: CreationInfo(author='us,usauto')
Magnitude
	    resource_id: ResourceIdentifier(id="smi:service.iris.edu/fdsnws/event/1/query?magnitudeid=216566657")
	            mag: 5.1
	 magnitude_type: 'Mww'
	  creation_info: CreationInfo(author='us')
